# Exact Diagonalization of a 1D Spin Chain
<b> Christina Lee

Category: Grad

Prerequisites: Many-Body Quantum Mechanics, Part 1 </b>

Check out part 1 for the theory background.  Today is all programming.

Here we will initialize how many spins we will have in our chain.  

Remember to keep a reasonable number.  

Your number of states goes as $2^n$, and the size of your Hamiltonian will goes as $4^n$.  A 10-spin chain will already have over a million entries, before taking into account our tricks.

In [1]:
n=4
nstates=2^n

16

## Representation of Spins in Binary

Since our spins are only up or down, we can represent those by 1 and 0.  If we had a spin 1 system, we could use trinary instead.  

By using the computer's binary representation of a number, we store states much more compactly

In [2]:
psi=collect(0:(nstates-1))
for p in psi
    println(bin(p,4),' ',p)
end

0000 0
0001 1
0010 2
0011 3
0100 4
0101 5
0110 6
0111 7
1000 8
1001 9
1010 10
1011 11
1100 12
1101 13
1110 14
1111 15


In [3]:
powers2=collect(0:(n-1))
powers2=2.^powers2
mask=[0;powers2]+[powers2;0]
mask=mask[2:end-1]
for i in 1:(length(mask))
    println(bin(mask[i],4))
end

0011
0110
1100


In [4]:
m=zeros(psi)
for i in 1:nstates
    m[i]=sum((psi[i]&powers2)./(powers2))
    #println(round(Int,(psi[i]&powers2)./powers2))
end

In [5]:
ind=sortperm(m)
m=(m[ind]-n/2)/2
psi=psi[ind]
[psi m]

16x2 Array{Float64,2}:
  0.0  -1.0
  1.0  -0.5
  2.0  -0.5
  4.0  -0.5
  8.0  -0.5
  3.0   0.0
  5.0   0.0
  6.0   0.0
  9.0   0.0
 10.0   0.0
 12.0   0.0
  7.0   0.5
 11.0   0.5
 13.0   0.5
 14.0   0.5
 15.0   1.0

In [6]:
psia=Array{Array{Int64}}(n+1)
old=1
new=1
for i in 1:(n+1)
    #println(old,' ',new)
    psia[i]=psi[old:new]
    old=new+1
    new=new+binomial(n,i)
end
psia

5-element Array{Array{Int64,N},1}:
 [0]            
 [1,2,4,8]      
 [3,5,6,9,10,12]
 [7,11,13,14]   
 [15]           

In [1]:
i=3
dim=binomial(n,i-1)
M=(i-1-n/2)*eye(Float64,dim,dim)

LoadError: LoadError: UndefVarError: n not defined
while loading In[1], in expression starting on line 2

In [8]:
test=psia[i]
for p in test
    println(bin(p,4),' ',p)
end

0011 3
0101 5
0110 6
1001 9
1010 10
1100 12


In [40]:
function findc(nonzero::Int,test::Array)
    bottom=1;
    diff=length(test);
    ii=ceil(Int,length(test)/2)
    found=false
    count=0
    while found==false && count < length(test)
        if test[ii]==nonzero
            return ii
        elseif nonzero<test[ii]
            diff=floor(Int,diff/2)
            ii=bottom+diff
            #println("less ",bottom,' ',ii,' ',diff,' ',test[ii])
            count+=1
        else 
            bottom=ii;
            diff=ceil(Int,diff/2);
            ii=bottom+diff
            #println("greater ",bottom,' ',ii,' ',diff,' ',test[ii])
            count+=1
        end
    end
    return 0
end

findc (generic function with 1 method)

In [45]:
for a in 1:length(test)
    temp=round(Int,(test[a]&powers2)./powers2)
    for b in 1:(n-1)
        if temp[b+1]!=temp[b]
            #println(temp,' ',test[a] $ mask[b], ' ',
            #round(Int,((test[a]$mask[b]) & powers2)./powers2))
            nonzero=test[a] $ mask[b]
            c=findc(nonzero,test)
            #println(c,' ',test[c])
            M[a,c]=.5
            M[c,a]=.5
        end
    end
end
eigvals(M)

6-element Array{Float64,1}:
 -1.11803    
 -0.5        
  0.0        
  1.06254e-16
  0.5        
  1.11803    